In [ ]:
import numpy as np
from utils import dataLoader, saliency, visualize
from model import NativeSpacemodel
import pandas as pd
import tensorflow as tf

### get a list of freesurfer processed files

In [ ]:
## filePaths - replace with your file paths - should point to brain.mgz
filePaths = ["dir\\fileLocation1\\brain.mgz",
             "dir\\fileLocation2\\brain.mgz"] 
## brain age csv save location
csvLocation = ""

csvName = "BAtest.csv"

In [ ]:
# limit TF memory usage
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print("Could not limit memory usage")

### load model and prepare data for prediction 

In [ ]:
BAPredictor = NativeSpacemodel.get_model()
modelWeights = "./model/NativeSpaceWeight.h5"
BAPredictor.load_weights(modelWeights)

brains = dataLoader.dataLoader(filePaths)

In [ ]:
[n, h, w, d] = np.shape(brains)
print("\Loaded ", n, " brains")

# make sure the prepocessed brains have 128^3 volumes  
assert h==w==d == 128

### preform prediction

In [ ]:
predictions = BAPredictor.predict(brains)

In [ ]:
dfBA = pd.DataFrame(predictions)
dfBA.columns = ["BA"]
dfBA.to_csv(csvLocation+csvName)

### Optional step : compute saliency maps

In [ ]:
sMaps = saliency.saliencyMap(BAPredictor, brains)
print(np.shape(sMaps), np.shape(brains))
#visualize.plot3Views(sMaps, slice1=64)

In [ ]:
# post-process saliency 
sMaps = saliency.postProcess(sMaps, brains)

In [ ]:
visualize.plot3Views(sMaps, slice1=64)